In [33]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC ,Ridge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split , KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt  
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

reading the files

In [96]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
train.shape , test.shape

((7000, 24), (3000, 23))

In [97]:
traintest = train.append(test)
traintest.shape

(10000, 24)

In [98]:
traintest.head(3)

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851


In [99]:
y = traintest['Attrition_rate']
traintest.drop(['Attrition_rate','Employee_ID'],axis=1,inplace=True)
traintest.head(3)

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,...,7.0,type2,3.0,4,0.7516,1.8688,2.0,4,5,3
1,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,...,6.0,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3
2,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,...,8.0,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3


In [100]:
traintest.shape , y.shape

((10000, 22), (10000,))

In [101]:
traintest.nunique(dropna=False)

Gender                        2
Age                          48
Education_Level               5
Relationship_Status           2
Hometown                      5
Unit                         12
Decision_skill_possess        4
Time_of_service              45
Time_since_promotion          5
growth_rate                  55
Travel_Rate                   3
Post_Level                    5
Pay_Scale                    11
Compensation_and_Benefits     5
Work_Life_balance             6
VAR1                          5
VAR2                          6
VAR3                          5
VAR4                          4
VAR5                          5
VAR6                          5
VAR7                          5
dtype: int64

In [102]:
traintest.isnull().sum()

Gender                         0
Age                          573
Education_Level                0
Relationship_Status            0
Hometown                       0
Unit                           0
Decision_skill_possess         0
Time_of_service              196
Time_since_promotion           0
growth_rate                    0
Travel_Rate                    0
Post_Level                     0
Pay_Scale                     12
Compensation_and_Benefits      0
Work_Life_balance             16
VAR1                           0
VAR2                         794
VAR3                           0
VAR4                         954
VAR5                           0
VAR6                           0
VAR7                           0
dtype: int64

for handling missing value in AGE

In [103]:
traintest.Age.value_counts(dropna=False).head(4)

NaN     573
22.0    347
27.0    336
25.0    320
Name: Age, dtype: int64

In [104]:
traintest.Age.describe()

count    9427.000000
mean       39.675507
std        13.574856
min        19.000000
25%        27.000000
50%        38.000000
75%        52.000000
max        65.000000
Name: Age, dtype: float64

In [105]:
#fig, ax = plt.subplots()
#ax.scatter(x = train['Age'], y = train['Attrition_rate'])
#plt.ylabel('Attrition_rate', fontsize=13)
#plt.xlabel('Age', fontsize=13)
#plt.show()

In [106]:
traintest['Age'].fillna(traintest.groupby(['Gender', 'Education_Level'])['Age'].transform('median'))
traintest.Age.describe()

count    9427.000000
mean       39.675507
std        13.574856
min        19.000000
25%        27.000000
50%        38.000000
75%        52.000000
max        65.000000
Name: Age, dtype: float64

In [107]:
traintest['Age'].fillna(traintest.groupby(['Gender', 'Time_since_promotion'])['Age'].transform('median'))
traintest.Age.describe()

count    9427.000000
mean       39.675507
std        13.574856
min        19.000000
25%        27.000000
50%        38.000000
75%        52.000000
max        65.000000
Name: Age, dtype: float64

In [108]:
mean_imputation_df = traintest.copy()
mean_imputation_df['Age_Mean_Filled'] = mean_imputation_df['Age'].fillna(traintest['Age'].mean())
mean_imputation_df.Age_Mean_Filled.describe()

count    10000.000000
mean        39.675507
std         13.180160
min         19.000000
25%         28.000000
50%         39.675507
75%         51.000000
max         65.000000
Name: Age_Mean_Filled, dtype: float64

In [109]:
traintest['Age'].fillna(traintest.groupby(['Gender', 'Hometown'])['Age'].transform('median'),inplace=True)
#traintest.Age.value_counts()

for handling missing value in var4 

In [110]:
traintest.VAR4.value_counts(dropna=False)

2.0    6369
1.0    1809
NaN     954
3.0     868
Name: VAR4, dtype: int64

In [111]:
traintest.VAR4.fillna(4.0,inplace=True)
traintest.VAR4.value_counts(dropna=False)

2.0    6369
1.0    1809
4.0     954
3.0     868
Name: VAR4, dtype: int64

for handling missing value in var2

In [112]:
traintest.VAR2.value_counts(dropna=False)

 0.7516    3582
-0.1048    2288
-0.9612    1469
-1.8176    1173
 NaN        794
 1.6081     694
Name: VAR2, dtype: int64

In [113]:
traintest.VAR2.fillna(0.0,inplace=True)
traintest.VAR2.value_counts(dropna=False)

 0.7516    3582
-0.1048    2288
-0.9612    1469
-1.8176    1173
 0.0000     794
 1.6081     694
Name: VAR2, dtype: int64

In [114]:
traintest.isnull().sum()

Gender                         0
Age                            0
Education_Level                0
Relationship_Status            0
Hometown                       0
Unit                           0
Decision_skill_possess         0
Time_of_service              196
Time_since_promotion           0
growth_rate                    0
Travel_Rate                    0
Post_Level                     0
Pay_Scale                     12
Compensation_and_Benefits      0
Work_Life_balance             16
VAR1                           0
VAR2                           0
VAR3                           0
VAR4                           0
VAR5                           0
VAR6                           0
VAR7                           0
dtype: int64

In [115]:
traintest['Pay_Scale'].fillna(traintest.Pay_Scale.median(),inplace=True)
traintest['Time_of_service'].fillna(traintest.Time_of_service.mean(),inplace=True)
traintest['Work_Life_balance'].fillna(traintest.Work_Life_balance.mean(),inplace=True)
traintest['time'] = traintest.apply(lambda row: row.Time_of_service - row.Time_since_promotion , axis = 1) 
traintest['age_while_joining'] = traintest.apply(lambda r:r.Age - r.Time_of_service,axis=1)
traintest.isnull().any().sum()

0

In [116]:
cat_features =  list(traintest.select_dtypes(include=object).columns)
for i in cat_features:
    l = LabelEncoder()
    traintest[i] = l.fit_transform(traintest[i])
traintest.shape

(10000, 24)

In [16]:
#cat_features =  list(traintest.select_dtypes(include=object).columns)
#traintest = pd.get_dummies(traintest,columns=cat_features,drop_first=True)

In [117]:
traintest.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,time,age_while_joining
0,0,42.0,4,0,1,2,2,4.0,4,33,...,3.0,4,0.7516,1.8688,2.0,4,5,3,0.0,38.0
1,1,24.0,3,1,3,3,0,5.0,4,36,...,4.0,3,-0.9612,-0.4537,2.0,3,5,3,1.0,19.0
2,0,58.0,3,0,0,8,2,27.0,3,51,...,1.0,4,-0.9612,-0.4537,3.0,3,8,3,24.0,31.0
3,0,26.0,3,1,2,1,1,4.0,3,56,...,1.0,3,-1.8176,-0.4537,4.0,3,7,3,1.0,22.0
4,0,31.0,1,0,3,3,2,5.0,4,62,...,3.0,1,0.7516,-0.4537,2.0,2,8,2,1.0,26.0


In [118]:
traintest.shape

(10000, 24)

In [19]:
main_X=traintest.copy()

In [20]:
sc = StandardScaler()
X = sc.fit_transform(traintest)

In [122]:
#X_train = X[:7000,:]
#Y_train = y[:7000]
#X_test = X[7000:,:]

X_train = traintest.iloc[:7000,:]
Y_train = y.iloc[:7000]
X_test = traintest.iloc[7000:,:]

print(X_train.shape,Y_train.shape)
print(X_test.shape)

(7000, 24) (7000,)
(3000, 24)


In [45]:
## Hyper Parameter Optimization

booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [3,5, 10,25,50]
learning_rate=[0.05,0.1,0.15,0.20,0.01]
min_child_weight=[0,1,2,4]
subsample =[0.5,0.75,1]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score,
    'subsample':subsample
    }

# XGBRegressor

In [41]:
%%time
regressor=xgb.XGBRegressor()
# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)
random_cv.fit(X_train,Y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  7.8min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 18.5min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 30.1min finished


Wall time: 30min 13s


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          r...
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                     

In [42]:
random_cv.best_params_

{'subsample': 0.5,
 'n_estimators': 500,
 'min_child_weight': 2,
 'max_depth': 5,
 'learning_rate': 0.2,
 'booster': 'gblinear',
 'base_score': 0.25}

In [129]:
xgbmodel = xgb.XGBRegressor(subsample= 0.5,
 n_estimators=700,
 min_child_weight= 2,
 max_depth= 5,
 learning_rate= 0.2,
 booster='gblinear',
 base_score= 0.25)
xgbmodel.fit(X_train,Y_train)

[00:56:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.25, booster='gblinear', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=-1, importance_type='gain', interaction_constraints=None,
             learning_rate=0.2, max_delta_step=None, max_depth=5,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=700, n_jobs=0, num_parallel_tree=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=0, scale_pos_weight=1, subsample=0.5, tree_method=None,
             validate_parameters=1, verbosity=None)

In [130]:
yxgbreg = xgbmodel.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = yxgbreg
sub.to_csv('xgboost_submission.csv',index=False)

In [64]:
#y_test = np.zeros((test.shape[0],1, 5))
#y_test.shape

(3000, 1, 5)

In [65]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
warnings.filterwarnings('ignore')
scores = []

i=0
for train_index, test_index in kfold.split(X_train):   
    x_train, x_test = X_train[train_index], X_train[test_index]
    y_train, y_test = Y_train[train_index], Y_train[test_index]

    xgb_model = xgb.XGBRegressor(subsample= 0.7,n_estimators=500,
                                 min_child_weight= 2,max_depth= 5,
                                 learning_rate= 0.2,booster='gblinear',
                                 base_score= 0.25)
    
    xgb_model.fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    scores.append(100*max(0,1-np.sqrt(mean_squared_error(y_test,y_pred))))
    y_test = xgb_model.predict(X_test,num_iteration = iter_)
    xgb_model.predict()
    
    
print(scores)
print('mean',np.mean(scores))
print('std',np.std(scores))

[20:18:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


(3000, 1)


ValueError: Can only tuple-index with a MultiIndex

In [88]:
n_folds=10 
seed =0
skf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
X, y = X_train, Y_train
    
preds = np.zeros(test.shape[0])
    
params={'subsample': 0.5,
 'n_estimators': 500,
 'min_child_weight': 2,
 'max_depth': 5,
 'learning_rate': 0.2,
 'booster': 'gblinear',
 'base_score': 0.25}
    
for i, (itr, icv) in enumerate(skf.split(X, y)):
    x_train, x_val = X[itr, :], X[icv, :]
    y_train, y_val = y[itr], y[icv]

    dtrain = xgb.DMatrix(data=x_train, label=y_train, missing=np.nan)
    dval = xgb.DMatrix(data = x_val, label=y_val, missing=np.nan)

    bst = xgb.train(
                params                = params,
                dtrain                = dtrain,
                num_boost_round       = 30_000,
                early_stopping_rounds = 200,
                evals                 = [(dtrain, 'train'), (dval, 'eval')],
                verbose_eval          = 500
            )

    score_, iter_ = bst.best_score, bst.best_iteration
    bst.predict()
    test_preds = bst.predict(xgb.DMatrix(X_train), num_iteration = iter_)
    preds += test_preds
preds=preds / n_folds
    

[23:50:04] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { max_depth, min_child_weight, n_estimators, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:0.18965	eval-rmse:0.20952
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 200 rounds.
Stopping. Best iteration:
[6]	train-rmse:0.18326	eval-rmse:0.20611



TypeError: predict() got an unexpected keyword argument 'num_iteration'

In [60]:
u=xgb_model.predict(X_test).reshape(3000,1)
u.shape

(3000, 1)

In [59]:
u=u.reshape(3000,1)
u.shape

(3000, 1)

In [135]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=42)
import lightgbm as lgb
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 80, 
    "max_bin": 512,
    "num_iterations": 600,
    "n_estimators": 1000
}

gbm = lgb.LGBMRegressor(**hyper_params)

gbm.fit(x_train, y_train,
        eval_set=[(x_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=100)
y_pr = gbm.predict(x_test)
print(100*max(0,1-np.sqrt(mean_squared_error(y_test,y_pr))))

[1]	valid_0's l1: 0.12549	valid_0's l2: 0.0348143
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.125485	valid_0's l2: 0.0348165
[3]	valid_0's l1: 0.125486	valid_0's l2: 0.0348196
[4]	valid_0's l1: 0.125479	valid_0's l2: 0.0348214
[5]	valid_0's l1: 0.125475	valid_0's l2: 0.0348226
[6]	valid_0's l1: 0.125473	valid_0's l2: 0.0348234
[7]	valid_0's l1: 0.125477	valid_0's l2: 0.0348253
[8]	valid_0's l1: 0.125477	valid_0's l2: 0.0348254
[9]	valid_0's l1: 0.125471	valid_0's l2: 0.0348265
[10]	valid_0's l1: 0.125465	valid_0's l2: 0.0348282
[11]	valid_0's l1: 0.125452	valid_0's l2: 0.0348256
[12]	valid_0's l1: 0.125438	valid_0's l2: 0.0348207
[13]	valid_0's l1: 0.12543	valid_0's l2: 0.034818
[14]	valid_0's l1: 0.125432	valid_0's l2: 0.0348191
[15]	valid_0's l1: 0.125417	valid_0's l2: 0.0348137
[16]	valid_0's l1: 0.125411	valid_0's l2: 0.0348119
[17]	valid_0's l1: 0.12541	valid_0's l2: 0.0348124
[18]	valid_0's l1: 0.125399	valid_0's l2: 0.0348089
[19]	valid_0's 

In [136]:
ylight = gbm.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ylight
sub.to_csv('light_submission.csv',index=False)

In [65]:
#accuracy = 81.24

# Cat boost

In [ ]:
from catboost import CatBoostRegressor
from ml_modules.custom_estimator import Estimator
from ml_modules.custom_fold_generator import FoldScheme

est = Estimator(model=CatBoostRegressor(eval_metric='RMSE', n_estimators=20000, od_type='Iter', 
                    od_wait=200, colsample_bylevel=0.7, max_depth=6, learning_rate=0.1),
              early_stopping_rounds=200, random_state=50, validation_scheme=FoldScheme.KFold,
              eval_metric='rmse', task_type='regression', scoring_metric=rmse, n_splits=5,
              categorical_features_indices=cat_indices)

train_preds = est.fit_transform(x_train, y_train)

In [126]:
cat = CatBoostRegressor(n_estimators = 20000,loss_function = 'RMSE',
                    eval_metric = 'RMSE',od_type='Iter', od_wait=200, colsample_bylevel=0.7, 
                          max_depth=6, learning_rate=0.1)

In [127]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=42)

cat.fit(x_train,y_train,use_best_model=True,eval_set=(x_test,y_test))

0:	learn: 0.1853018	test: 0.1865798	best: 0.1865798 (0)	total: 3.25ms	remaining: 1m 5s
1:	learn: 0.1851527	test: 0.1865115	best: 0.1865115 (1)	total: 6.92ms	remaining: 1m 9s
2:	learn: 0.1849283	test: 0.1864992	best: 0.1864992 (2)	total: 10.5ms	remaining: 1m 10s
3:	learn: 0.1846529	test: 0.1865960	best: 0.1864992 (2)	total: 14.2ms	remaining: 1m 10s
4:	learn: 0.1844555	test: 0.1865584	best: 0.1864992 (2)	total: 17.7ms	remaining: 1m 10s
5:	learn: 0.1842959	test: 0.1865597	best: 0.1864992 (2)	total: 21.2ms	remaining: 1m 10s
6:	learn: 0.1839314	test: 0.1865902	best: 0.1864992 (2)	total: 25.6ms	remaining: 1m 13s
7:	learn: 0.1837677	test: 0.1865948	best: 0.1864992 (2)	total: 29.2ms	remaining: 1m 12s
8:	learn: 0.1835603	test: 0.1865948	best: 0.1864992 (2)	total: 32.5ms	remaining: 1m 12s
9:	learn: 0.1832971	test: 0.1866367	best: 0.1864992 (2)	total: 36ms	remaining: 1m 12s
10:	learn: 0.1831170	test: 0.1866253	best: 0.1864992 (2)	total: 39.7ms	remaining: 1m 12s
11:	learn: 0.1829644	test: 0.186601

110:	learn: 0.1666882	test: 0.1879629	best: 0.1864992 (2)	total: 520ms	remaining: 1m 33s
111:	learn: 0.1665043	test: 0.1880088	best: 0.1864992 (2)	total: 525ms	remaining: 1m 33s
112:	learn: 0.1663423	test: 0.1880023	best: 0.1864992 (2)	total: 529ms	remaining: 1m 33s
113:	learn: 0.1661866	test: 0.1879974	best: 0.1864992 (2)	total: 535ms	remaining: 1m 33s
114:	learn: 0.1660630	test: 0.1880350	best: 0.1864992 (2)	total: 538ms	remaining: 1m 33s
115:	learn: 0.1659502	test: 0.1880109	best: 0.1864992 (2)	total: 544ms	remaining: 1m 33s
116:	learn: 0.1659197	test: 0.1880031	best: 0.1864992 (2)	total: 550ms	remaining: 1m 33s
117:	learn: 0.1657993	test: 0.1880454	best: 0.1864992 (2)	total: 553ms	remaining: 1m 33s
118:	learn: 0.1657430	test: 0.1880440	best: 0.1864992 (2)	total: 557ms	remaining: 1m 33s
119:	learn: 0.1656204	test: 0.1880872	best: 0.1864992 (2)	total: 563ms	remaining: 1m 33s
120:	learn: 0.1655358	test: 0.1881069	best: 0.1864992 (2)	total: 567ms	remaining: 1m 33s
121:	learn: 0.1654602

In [128]:
ycat = cat.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ycat
sub.to_csv('cat_submission.csv',index=False)

In [138]:
ens = ylight*0.6 +ycat*0.2 +yxgbreg*0.2
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ens
sub.to_csv('ensemble_submission.csv',index=False)

# XGBRFRegressor 

In [50]:
from xgboost import XGBRFRegressor
regressor=XGBRFRegressor()

ImportError: cannot import name 'XGBRFRegressor'

In [ ]:
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=60,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)
random_cv.fit(X_train,Y_train)

# Ridge 

In [154]:
%%time
ridge=Ridge()
parameters={'alpha':[1e-10,1e-8,1e-3,1,5,10,20,30,45,55,100,110,120,130,150,1000,2000,4000,6000,7000,8000,9000]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=10)
ridge_regressor.fit(X_train,Y_train)

Wall time: 2.08 s


In [155]:
ridge_regressor.best_params_

{'alpha': 8000}

In [156]:
yridge = ridge_regressor.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = yridge
sub.to_csv('ridge_submission.csv',index=False)

In [78]:
#accuracy = 81.28

# Lasso

In [159]:
%%time
lasso=Lasso()
parameters={'alpha':[1e-10,1e-8,1e-4,1e-3,1e-2,1,5,10,20,30,35,40,55,100]}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=12)

lasso_regressor.fit(X_train,Y_train)

Wall time: 7.54 s


In [160]:
lasso_regressor.best_params_

{'alpha': 0.001}

In [161]:
ylasso = lasso_regressor.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ylasso
sub.to_csv('lasso_submission.csv',index=False)

#accu = 81.24

# ElasticNet

In [88]:
enet=ElasticNet()
parameters={'alpha':[1e-8,1e-3,1e-2,0,1,2,3,4,5,10,20,30,35,40],
           'l1_ratio':[0.2,0.5,0.8,1],
           'max_iter':[1000,500,1500]}
elasticnet_regressor=GridSearchCV(enet,parameters,
                                  scoring='neg_mean_squared_error',cv=5)

elasticnet_regressor.fit(X_train,Y_train)

C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 97.23536916662368, tolerance: 0.019589481083785536
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\DELL

C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 97.13042393452763, tolerance: 0.019632686899999998
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95.3800766676157, tolerance: 0.019246960757474285
  positive)
C

C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 96.50424971300927, tolerance: 0.019496539782949816
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92.42912701782072, tolerance: 0.01862676156024
  positive)
C:\U

C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 97.13042393452757, tolerance: 0.019632686899999998
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95.38007666761563, tolerance: 0.019246960757474285
  positive)


C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 96.50424971300926, tolerance: 0.019496539782949816
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\DELL\Miniconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92.4291270178207, tolerance: 0.01862676156024
  positive)
C:\Us

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1e-08, 0.001, 0.01, 0, 1, 2, 3, 4, 5, 10, 20,
                                   30, 35, 40],
                         'l1_ratio': [0.2, 0.5, 0.8, 1],
                         'max_iter': [1000, 500, 1500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [89]:
elasticnet_regressor.best_params_

{'alpha': 0.01, 'l1_ratio': 1, 'max_iter': 1000}

In [90]:
ypred = elasticnet_regressor.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ypred
sub.to_csv('elastic_submission.csv',index=False)

# RandomForest

In [57]:
random_grid = {'n_estimators': [100,200, 400, 600, 800, 1000, 1200, 1400,1800],
               'max_features':['auto', 'sqrt','log2'],
               'max_depth': [7,10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
               'min_samples_split':  [2, 5, 10],
               'min_samples_leaf': [1, 2, 4,10],
               'bootstrap':[True, False]}

In [58]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 50, cv = 3, verbose=2, 
                               random_state=42,n_jobs = -1)
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 19.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [59]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'max_features': 'sqrt',
 'max_depth': 7,
 'bootstrap': True}

In [60]:
'''rf =RandomForestRegressor(n_estimators=800,
                          min_samples_split=10,max_features='log2',
                         min_samples_leaf=10,max_depth=7,
                         bootstrap=True)'''
rf = RandomForestRegressor(n_estimators=100,min_samples_leaf=10,
                          min_samples_split=2,max_features='sqrt',
                          max_depth=7,bootstrap=True)
rf.fit(X_train,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [62]:
yrf = rf.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = yrf
sub.to_csv('randomforest_submission.csv',index=False)


#accuracy = 81.225
#accuracy = 81.20658

In [105]:
ypred= rf_random.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ypred
sub.to_csv('randomforest_submission.csv',index=False)

In [116]:
ensemble = ylasso*0.90 +yridge*.10

sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ensemble
sub.to_csv('ensemble_submission.csv',index=False)

#accuracy = 81.2612

# GradientBoostingRegressor

In [65]:
%%time

random_grid = {'n_estimators': [90,100,200, 400, 600, 800, 1000, 1200, 1400],
               'loss': ['ls', 'lad', 'huber', 'quantile'],
               'max_depth': [3,7,10, 20, 30, 50, 60, 80, 90, 100, None],
               'min_samples_split':  [2, 5, 10],
               'min_samples_leaf': [1, 2, 4,10],
               'learning_rate': [0.01,0.1,0.2,0.02]
              }
grad = GradientBoostingRegressor()
rf_random = RandomizedSearchCV(estimator = grad, 
                               param_distributions = random_grid, 
                               n_iter = 50, cv = 3, verbose=1, 
                               random_state=42,n_jobs = -1)
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 69.7min finished


Wall time: 1h 9min 48s


In [66]:
rf_random.best_params_

{'n_estimators': 90,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'max_depth': 7,
 'loss': 'ls',
 'learning_rate': 0.01}

In [70]:
grad = GradientBoostingRegressor(n_estimators=500,min_samples_split=2,
                                min_samples_leaf=10,max_depth=7,
                                loss='ls',learning_rate=0.01,verbose=1)
grad.fit(X_train,Y_train)

      Iter       Train Loss   Remaining Time 
         1           0.0344           27.14s
         2           0.0344           25.48s
         3           0.0343           25.16s
         4           0.0343           25.71s
         5           0.0342           25.74s
         6           0.0342           27.77s
         7           0.0341           27.41s
         8           0.0341           27.19s
         9           0.0340           26.85s
        10           0.0340           26.71s
        20           0.0336           26.05s
        30           0.0332           24.48s
        40           0.0328           23.48s
        50           0.0324           23.67s
        60           0.0320           22.79s
        70           0.0317           22.00s
        80           0.0313           21.35s
        90           0.0310           20.87s
       100           0.0307           20.23s
       200           0.0282           14.27s
       300           0.0263            8.91s
       40

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.01, loss='ls', max_depth=7,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=10, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [73]:
ygrad= grad.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ygrad
sub.to_csv('grad_submission.csv',index=False)

#80.65

# BayesianRidge

In [164]:
param = { 'n_iter':[200,400,100,150,500],
          'lambda_1':[1e-06,1e-04,1e-02,1e-01,0],
          'alpha_1': [1e-06,1e-04,1e-02,1e-01,0],
          'lambda_2':[1e-06,1e-04,1e-02,1e-01,0],
          'alpha_2': [1e-06,1e-04,1e-02,1e-01,0]}
bay = BayesianRidge()
rf_random = RandomizedSearchCV(estimator = bay, 
                               param_distributions = param, 
                               n_iter = 100, cv = 10, verbose=0.5, 
                               random_state=42,n_jobs = -1)
rf_random.fit(X_train,Y_train)
print(rf_random.best_params_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:    2.3s


{'n_iter': 100, 'lambda_2': 0.0001, 'lambda_1': 0.1, 'alpha_2': 0.1, 'alpha_1': 1e-06}


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.6s finished


In [167]:
bay = BayesianRidge(n_iter= 100, lambda_2= 1e-04, 
                    lambda_1= 0.1, alpha_2=0.1, alpha_1= 1e-6,verbose=2,normalize=True,compute_score=True)
bay.fit(X_train,Y_train)

#accuarcy default = 81.26082


Convergence after  14  iterations


BayesianRidge(alpha_1=1e-06, alpha_2=0.1, compute_score=True, copy_X=True,
              fit_intercept=True, lambda_1=0.1, lambda_2=0.0001, n_iter=100,
              normalize=True, tol=0.001, verbose=2)

In [168]:
ybay = bay.predict(X_test)

sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ybay
sub.to_csv('bay_submission.csv',index=False)

In [88]:
ensemble = ybay*0.60 +yridge*.20 + ylasso*.20
 
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ybay
sub.to_csv('bay_submission.csv',index=False)


In [90]:
from sklearn.neural_network import MLPRegressor
MLP = MLPRegressor()

In [91]:
MLP.fit(X_train,Y_train)
ymlp = MLP.predict(X_test)

sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ymlp
sub.to_csv('mlp_submission.csv',index=False)


In [181]:
n_folds = 5

from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train, Y_train, 
                                   scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [182]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [183]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [184]:
averaged_models = AveragingModels(models = (ridge ,bay, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Convergence after  29  iterations
Convergence after  11  iterations
Convergence after  16  iterations
Convergence after  15  iterations
Convergence after  26  iterations
 Averaged base models score: 0.1856 (0.0069)



In [186]:
stacked_averaged_models = StackingAveragedModels(base_models = (bay,ridge),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

KeyError: "None of [Int64Index([   1,    2,    3,    4,    5,    7,    8,    9,   11,   12,\n            ...\n            5582, 5583, 5584, 5586, 5591, 5592, 5593, 5594, 5595, 5597],\n           dtype='int64', length=4480)] are in the [columns]"